**** _Introductory paragraph_ ****

Requests is python's go to package for making api calls.

In [1]:
import requests
import pandas as pd

Let's make a request and store it as a response object (more info can be found on response objects here). I'm going to be using the business licences dataset on the City of Vancouver's Open Data portal. For now, we'll just use their dataset search:

In [104]:
response = requests.get('https://opendata.vancouver.ca/api/records/1.0')

In [105]:
response.status_code

404

The 404 response tells us that the program couldn't find what we were looking for. Turns out the above is just the base for the whole CoV api. We need to give the request a little more information.

In [111]:
payload = {
    'dataset': 'business-licences',
    #'rows': 100
    #'start': 1
}
response = requests.get('https://opendata.vancouver.ca/api/records/1.0/download/', params=payload)
response.status_code

200

In [98]:
payload = {
    'dataset': 'business-licences',
    'rows': 10,
    'start': 0
}


def clean_coordinates(field_dict):
    if 'geom' in field_dict.keys():
        field_dict['x_coord'] = field_dict['geom']['coordinates'][0]
        field_dict['y_coord'] = field_dict['geom']['coordinates'][1]
        del field_dict['geom']
        
    return field_dict

l = []
while payload['start']<100:
    response = requests.get('https://opendata.vancouver.ca/api/records/1.0/search/', params=payload)
    records = response.json()['records']
    fields = [clean_coordinates(f['fields']) for f in records]
    l.extend(fields)
    payload['start'] += 1000

Sweet, we're in business. Let's see what the request gave us (Hint: if you want the content as a string - here it's in bytes - use the `text` method instead of the `content` method

In [100]:
bl_data = pd.DataFrame.from_records(l)

In [103]:
bl_data.columns

Index(['province', 'expireddate', 'house', 'licencerevisionnumber', 'street',
       'unit', 'city', 'businesstype', 'businesssubtype', 'unittype',
       'licencersn', 'status', 'extractdate', 'numberofemployees', 'localarea',
       'folderyear', 'businessname', 'postalcode', 'issueddate', 'country',
       'feepaid', 'businesstradename', 'licencenumber', 'x_coord', 'y_coord'],
      dtype='object')

Okay, that's pretty legible. We've got the business name, location, coordinates, neighborhood, license number, expiry date... All sorts of useful information. Now how do we get all of this into a database?

In [33]:
records[0]['fields']

{'province': 'BC',
 'expireddate': '2013-12-31',
 'house': '966',
 'licencerevisionnumber': '00',
 'street': 'SW MARINE DRIVE',
 'unit': '60',
 'city': 'Vancouver',
 'businesstype': 'Manufacturer',
 'businesssubtype': 'Garments',
 'unittype': 'Unit',
 'licencersn': '1798226',
 'geom': {'type': 'Point',
  'coordinates': [-123.128122611431, 49.2050691574352]},
 'status': 'Issued',
 'extractdate': '2019-07-21T20:49:07+00:00',
 'numberofemployees': '5',
 'localarea': 'Marpole',
 'folderyear': '13',
 'businessname': '0755089 BC Ltd',
 'postalcode': 'V6P 5Z2',
 'issueddate': '2013-01-26T22:55:45+00:00',
 'country': 'CA',
 'feepaid': 335,
 'businesstradename': 'Embroidme Vancouver',
 'licencenumber': '13-178807'}

In [27]:
set([d['fields']['folderyear'] for d in records])

{'20'}

In [27]:
dates = []
for i in range(len(records)):
    print(records[i]['fields']['expireddate'])

2013-12-31


KeyError: 'expireddate'

In [29]:
records[1]['fields']

{'status': 'Pending',
 'city': 'Vancouver',
 'businesstype': 'Manufacturer',
 'extractdate': '2019-07-21T20:49:07+00:00',
 'numberofemployees': '2',
 'house': '614',
 'country': 'CA',
 'localarea': 'Strathcona',
 'folderyear': '13',
 'businesssubtype': 'Other',
 'licencerevisionnumber': '00',
 'businesstradename': 'Saul Good Gift Co',
 'province': 'BC',
 'street': 'ALEXANDER ST',
 'licencersn': '1798227',
 'businessname': 'Saul Good Gift Co Inc',
 'postalcode': 'V6A 1C9',
 'geom': {'type': 'Point',
  'coordinates': [-123.090879404489, 49.2836676278532]},
 'licencenumber': '13-178808'}